# GroupBy

In [2]:
from google.cloud import bigquery
client = bigquery.Client.from_service_account_json("./My Project 61217-9183df10355d.json")
dataset_ref = client.dataset('hacker_news', project = 'bigquery-public-data')
# API request -fetch dataset
dataset = client.get_dataset(dataset_ref)

In [3]:
# list all the tables in the hacker news dataset
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

comments
full
full_201510
stories


In [4]:
table_ref = dataset_ref.table('comments')
table = client.get_table(table_ref)

In [6]:
client.list_rows(table, max_results=5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


Let's use the table to see which comments generated the most replies. Since:

the parent column indicates the comment that was replied to, and
the id column has the unique ID used to identify each comment,
we can GROUP BY the parent column and COUNT() the id column in order to figure out the number of comments that were made as responses to a specific comment. (This might not make sense immediately -- take your time here to ensure that everything is clear!)

Furthermore, since we're only interested in popular comments, we'll look at comments with more than ten replies. So, we'll only return groups HAVING more than ten ID's.

In [8]:
query = """
        SELECT parent, COUNT(id)
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY parent
        HAVING COUNT(id)>10
        """

In [10]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()
popular_comments.head()

,parent,f0_
0,6427895,46
1,202918,44
2,8120079,148
3,9016949,38
4,7075537,51


### Aliasing

In [11]:
# Improved version of earlier query, now with aliasing & improved readability
query = """
        SELECT parent, COUNT(1) AS NumPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY parent
        HAVING  COUNT(1) >10
        
        ""

In [14]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improved_df = query_job.to_dataframe()

# Print the first five rows of the DataFrame
improved_df.head()

BadRequest: 400 Unrecognized name: i at [2:30]

(job ID: 6e900244-90ff-4cd5-b35b-aa573a40bacc)

               -----Query Job SQL Follows-----               

    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:        SELECT parent, COUNT(i) AS NumPosts
   3:        FROM `bigquery-public-data.hacker_news.comments`
   4:        GROUP BY parent
   5:        HAVING  COUNT(1) >10
   6:        
   7:        
    |    .    |    .    |    .    |    .    |    .    |

In [15]:
# Group By

query = """

        SELECT parent, author, COUNT(id)
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY parent
    
        """

In [16]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)
query_job.to_dataframe()

BadRequest: 400 SELECT list expression references column author which is neither grouped nor aggregated at [3:24]

(job ID: 863a8baa-49b3-4e34-bb63-33c85415b4d5)

               -----Query Job SQL Follows-----               

    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:
   3:        SELECT parent, author, COUNT(id)
   4:        FROM `bigquery-public-data.hacker_news.comments`
   5:        GROUP BY parent
   6:    
   7:        
    |    .    |    .    |    .    |    .    |    .    |